In [1]:
# Dependencies
import json
import requests
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
filepath = 'hate_crime.csv'
df = pd.read_csv(filepath, low_memory=False)
df.head()

,INCIDENT_ID,DATA_YEAR,ORI,PUB_AGENCY_NAME,PUB_AGENCY_UNIT,AGENCY_TYPE_NAME,STATE_ABBR,STATE_NAME,DIVISION_NAME,REGION_NAME,...,OFFENDER_RACE,OFFENDER_ETHNICITY,VICTIM_COUNT,OFFENSE_NAME,TOTAL_INDIVIDUAL_VICTIMS,LOCATION_NAME,BIAS_DESC,VICTIM_TYPES,MULTIPLE_OFFENSE,MULTIPLE_BIAS
0,3015,1991,AR0040200,Rogers,NaN,City,AR,Arkansas,West South Central,South,...,White,NaN,1,Intimidation,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-Black or African American,Individual,S,S
1,3016,1991,AR0290100,Hope,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,1,Simple Assault,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,S,S
2,43,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,1,Aggravated Assault,1.0,Residence/Home,Anti-Black or African American,Individual,S,S
3,44,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,2,Aggravated Assault;Destruction/Damage/Vandalis...,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,M,S
4,3017,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,1,Aggravated Assault,1.0,Service/Gas Station,Anti-White,Individual,S,S


In [3]:
filtered_df = df[['DATA_YEAR', 'STATE_ABBR', 'LOCATION_NAME', 'OFFENSE_NAME', 'MULTIPLE_OFFENSE', 'VICTIM_TYPES', 
                  'BIAS_DESC', 'OFFENDER_RACE']]
filtered_df.head()

,DATA_YEAR,STATE_ABBR,LOCATION_NAME,OFFENSE_NAME,MULTIPLE_OFFENSE,VICTIM_TYPES,BIAS_DESC,OFFENDER_RACE
0,1991,AR,Highway/Road/Alley/Street/Sidewalk,Intimidation,S,Individual,Anti-Black or African American,White
1,1991,AR,Highway/Road/Alley/Street/Sidewalk,Simple Assault,S,Individual,Anti-White,Black or African American
2,1991,AR,Residence/Home,Aggravated Assault,S,Individual,Anti-Black or African American,Black or African American
3,1991,AR,Highway/Road/Alley/Street/Sidewalk,Aggravated Assault;Destruction/Damage/Vandalis...,M,Individual,Anti-White,Black or African American
4,1991,AR,Service/Gas Station,Aggravated Assault,S,Individual,Anti-White,Black or African American


In [4]:
years_df = filtered_df[(filtered_df['DATA_YEAR'] >= 2014) & (df['DATA_YEAR'] <= 2018)]
years_df.head()

,DATA_YEAR,STATE_ABBR,LOCATION_NAME,OFFENSE_NAME,MULTIPLE_OFFENSE,VICTIM_TYPES,BIAS_DESC,OFFENDER_RACE
169146,2014,AK,Residence/Home,Destruction/Damage/Vandalism of Property,S,Individual,Anti-Gay (Male),Unknown
169147,2014,AK,Parking/Drop Lot/Garage,Aggravated Assault,S,Individual,Anti-American Indian or Alaska Native,White
169148,2014,AK,Shelter-Mission/Homeless,Simple Assault,S,Individual,Anti-Black or African American,White
169149,2014,AK,Parking/Drop Lot/Garage,Robbery,S,Individual,Anti-American Indian or Alaska Native,Black or African American
169150,2014,AK,Parking/Drop Lot/Garage,Simple Assault,S,Individual,Anti-American Indian or Alaska Native,White


In [5]:
years_df['OFFENSE_NAME'].value_counts()

Destruction/Damage/Vandalism of Property                                                    9025
Intimidation                                                                                7922
Simple Assault                                                                              6975
Aggravated Assault                                                                          3278
Robbery                                                                                      597
                                                                                            ... 
Impersonation;Intimidation                                                                     1
False Pretenses/Swindle/Confidence Game;Intimidation                                           1
Purchasing Prostitution                                                                        1
Arson;Destruction/Damage/Vandalism of Property;Motor Vehicle Theft                             1
Aggravated Assault;Burglary/Br

In [6]:
year14 = years_df.loc[years_df['DATA_YEAR'] == 2014,:]['OFFENSE_NAME'].value_counts()
year14

Destruction/Damage/Vandalism of Property                                                1647
Intimidation                                                                            1321
Simple Assault                                                                          1257
Aggravated Assault                                                                       551
Burglary/Breaking & Entering                                                             140
                                                                                        ... 
Arson;Burglary/Breaking & Entering                                                         1
Destruction/Damage/Vandalism of Property;Theft of Motor Vehicle Parts or Accessories       1
Kidnapping/Abduction                                                                       1
Aggravated Assault;Destruction/Damage/Vandalism of Property;Kidnapping/Abduction           1
Theft From Coin-Operated Machine or Device                            

In [7]:
year15 = years_df.loc[years_df['DATA_YEAR'] == 2015,:]['OFFENSE_NAME'].value_counts()
year15

Destruction/Damage/Vandalism of Property                                                       1620
Intimidation                                                                                   1432
Simple Assault                                                                                 1381
Aggravated Assault                                                                              625
Burglary/Breaking & Entering                                                                    123
                                                                                               ... 
Intimidation;Robbery                                                                              1
Burglary/Breaking & Entering;Destruction/Damage/Vandalism of Property;Simple Assault              1
Aggravated Assault;Burglary/Breaking & Entering;Weapon Law Violations                             1
All Other Larceny;Destruction/Damage/Vandalism of Property;Robbery;Stolen Property Offenses       1


In [8]:
year16 = years_df.loc[years_df['DATA_YEAR'] == 2016,:]['OFFENSE_NAME'].value_counts()
year16

Destruction/Damage/Vandalism of Property                                                1812
Intimidation                                                                            1528
Simple Assault                                                                          1413
Aggravated Assault                                                                       628
Robbery                                                                                  114
                                                                                        ... 
Aggravated Assault;Destruction/Damage/Vandalism of Property;Stolen Property Offenses       1
Motor Vehicle Theft;Sodomy                                                                 1
Sexual Assault With An Object;Sodomy                                                       1
All Other Larceny;Simple Assault                                                           1
Intimidation;Weapon Law Violations                                    

In [9]:
year17 = years_df.loc[years_df['DATA_YEAR'] == 2017,:]['OFFENSE_NAME'].value_counts()
year17

Destruction/Damage/Vandalism of Property                                                2223
Intimidation                                                                            1754
Simple Assault                                                                          1377
Aggravated Assault                                                                       738
Robbery                                                                                  141
                                                                                        ... 
Arson;Burglary/Breaking & Entering;Destruction/Damage/Vandalism of Property                1
All Other Larceny;Destruction/Damage/Vandalism of Property;Theft From Motor Vehicle        1
Arson;Murder and Nonnegligent Manslaughter                                                 1
Destruction/Damage/Vandalism of Property;Motor Vehicle Theft                               1
Destruction/Damage/Vandalism of Property;Theft of Motor Vehicle Parts 

In [10]:
year18 = years_df.loc[years_df['DATA_YEAR'] == 2018,:]['OFFENSE_NAME'].value_counts()
year18

Intimidation                                                                          1887
Destruction/Damage/Vandalism of Property                                              1723
Simple Assault                                                                        1547
Aggravated Assault                                                                     736
Drug/Narcotic Violations                                                               138
                                                                                      ... 
Intimidation;Theft From Motor Vehicle                                                    1
Burglary/Breaking & Entering;Destruction/Damage/Vandalism of Property;Intimidation       1
Burglary/Breaking & Entering;Theft From Motor Vehicle                                    1
Credit Card/Automated Teller Machine Fraud;Motor Vehicle Theft                           1
Robbery;Shoplifting                                                                      1